In [1]:
# Testing the downloading data method from:
# https://claut.gitlab.io/man_ccia/lab2.html
# Import the required modules
from pyesgf.search import SearchConnection
import os
import pandas as pd
import requests
from tqdm import tqdm

# Set the os environment to on
os.environ['ESGF_PYCLIENT_NO_FACETS_STAR_WARNING'] = "on"

In [2]:
# Import the functions
from testing_download_functions import query_data_esgf, extract_file_context, \
                                        download_file, extract_file_context_multithread

In [3]:
# set the search connection
# to the LLNL search node
conn = SearchConnection('https://esgf-node.llnl.gov/esg-search', distrib=True)

In [4]:
print(type(conn))

<class 'pyesgf.search.connection.SearchConnection'>


In [5]:
# test the function for querying the database
results = query_data_esgf(conn,
                        source_id='CanESM5',
                        experiment_id='hist-aer',
                        variable_id='tas',
                        table_id='Amon',
                        data_node='crd-esgf-drc.ec.gc.ca',)

# print the len of the results
print(len(results))

# print the type of the results
print(type(results))

# print the results
print(results)

30
<class 'pyesgf.search.results.ResultSet'>


In [6]:
# Print the details of the first result
print(results[0].json['id'])

CMIP6.DAMIP.CCCma.CanESM5.hist-aer.r10i1p1f1.Amon.tas.gn.v20190429|crd-esgf-drc.ec.gc.ca


In [7]:
# # Use the function to extract the file context
# files_list_mt = extract_file_context_multithread(results)

# # Turn the list into a dataframe
# files_df_mt = pd.DataFrame.from_dict(files_list_mt)

# files_df_mt

In [8]:
# Extract the file context
files_list = extract_file_context(results)

# # Turn the list into a dataframe
# files_df = pd.DataFrame.from_dict(files_list)

# files_df

Extracting file context for 30 datasets...


In [9]:
# Print the type of the files list
print(type(files_list))

# Extract this into a dataframe
files_df = pd.DataFrame.from_dict(files_list)
files_df

# Assert that all filenames contrain the string "185001" and "202012"
assert all(files_df['filename'].str.contains('185001')), "Not all filenames contain the string 185001"
assert all(files_df['filename'].str.contains('202012')), "Not all filenames contain the string 202012"

files_df

<class 'list'>


,filename,url
0,tas_Amon_CanESM5_hist-aer_r10i1p1f1_gn_185001-...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
1,tas_Amon_CanESM5_hist-aer_r1i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
2,tas_Amon_CanESM5_hist-aer_r4i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
3,tas_Amon_CanESM5_hist-aer_r2i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
4,tas_Amon_CanESM5_hist-aer_r7i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
5,tas_Amon_CanESM5_hist-aer_r9i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
6,tas_Amon_CanESM5_hist-aer_r6i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
7,tas_Amon_CanESM5_hist-aer_r8i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
8,tas_Amon_CanESM5_hist-aer_r5i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...
9,tas_Amon_CanESM5_hist-aer_r3i1p1f1_gn_185001-2...,http://crd-esgf-drc.ec.gc.ca/thredds/fileServe...


In [10]:
# Set up the directory to download to
download_dir = "/gws/nopw/j04/scenario/users/benhutch/DAMIP"

# Set up the variable
variable = 'tas'

# Set up the experiment id
experiment_id = 'hist-aer'

# Set up the model
model = 'CanESM5'

# Set up the directory
download_path = os.path.join(download_dir, experiment_id, 
                             variable, model)

# Print the download path
print(download_path)

# Use the download function to download a single file
download_file(files_df['url'][0], 
              files_df['filename'][0], download_path)

/gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5
Saving to /gws/nopw/j04/scenario/users/benhutch/DAMIP/hist-aer/tas/CanESM5tas_Amon_CanESM5_hist-aer_r10i1p1f1_gn_185001-202012.nc


53.1kKiB [00:05, 9.87kKiB/s]                          


FileNotFoundError: [Errno 2] No such file or directory: 'tas_Amon_CanESM5_hist-aer_r10i1p1f1_gn_185001-202012.nc'